In [2]:
from ximea import xiapi
import cv2
import time
import os
from PIL import Image
import numpy as np
### runn this command first echo 0|sudo tee /sys/module/usbcore/parameters/usbfs_memory_mb  ###
 
#create instance for first connected camera
cam = xiapi.Camera()

In [ ]:
#start communication
#to open specific device, use:
#cam.open_device_by_SN('41305651')
#(open by serial number)
print('Opening first camera...')
cam.open_device()
 
#settings
cam.set_exposure(10000)
cam.set_param('imgdataformat','XI_RGB32')
cam.set_param('auto_wb', 1)
print('Exposure was set to %i us' %cam.get_exposure())
 
#create instance of Image to store image data and metadata
img = xiapi.Image()
 
#start data acquisition
print('Starting data acquisition...')
cam.start_acquisition()
 
i = 0
 
while cv2.waitKey(1) != ord('q'):
    cam.get_image(img)
    image = img.get_image_data_numpy()
    image = cv2.resize(image,(1024,1024))

    if cv2.waitKey(1) == ord('s'):
        cv2.imwrite(f"saved_imgs/img{i}.jpg", image)
        i += 1

    cv2.imshow("test2", image)

 
 
 
 
#stop data acquisition
print('Stopping acquisition...')
cam.stop_acquisition()
 
#stop communication
cam.close_device()
cv2.destroyAllWindows()
print('Done.')

In [29]:
def mosaic_2x2(imgs):
    for i in range(len(imgs)):
        imgs[i] = cv2.resize(imgs[i], (360,360))
    horizontal = np.concatenate(imgs[:2], axis=0)
    horizontal_2 = np.concatenate(imgs[2:], axis=0)

    mosaic = np.concatenate((horizontal, horizontal_2), axis=1)

    return mosaic

In [30]:
import cv2
import os
from PIL import Image
import numpy as np

imgs = []

# Define the directory containing the images
image_directory = 'saved_imgs'

# List all files in the directory
files = os.listdir(image_directory)

# Load images
images = []
for image_file in files:
    image_path = os.path.join(image_directory, image_file)
    image = cv2.imread(image_path)
    images.append(image)

# mosaic = mosaic_2x2(images)

# cv2.imshow("top 4 people living on the Earth", mosaic)
# cv2.waitKey(0)

In [33]:
# manually applying kernel to the top-left image

mosaic = mosaic_2x2(images)
img = mosaic[:mosaic.shape[0]//2, :mosaic.shape[1]//2]

shape = (img.shape[0], img.shape[1], img.shape[2])
new_img = np.empty(shape, dtype=np.int32)

kernel =  np.array([[0, -1, 0],       
                    [-1, 5, -1],       
                    [0, -1, 0]],  dtype=np.int32)


iterate = img.shape[1]//kernel.shape[1]

for i in range(iterate):
    for j in range(iterate):
        slice = img[i*3 : (i*3)+3, j*3 : (j*3)+3]
        for z in range(3):
            new_img[i*3 : (i*3)+3, j*3 : (j*3)+3][z] = slice[z] * kernel[z]


mosaic[:mosaic.shape[0]//2, :mosaic.shape[1]//2] = new_img


cv2.imshow("top 4 people living on the Earth", mosaic)
cv2.waitKey(0)

113

In [35]:
# Automatic kernel application

mosaic = mosaic_2x2(images)
img = mosaic[:mosaic.shape[0]//2, :mosaic.shape[1]//2]

kernel =  np.array([[0, -1, 0],       
                    [-1, 5, -1],       
                    [0, -1, 0]],  dtype=np.int32)

new_img = cv2.filter2D(src=img, ddepth=-1, kernel=kernel)

mosaic[:mosaic.shape[0]//2, :mosaic.shape[1]//2] = new_img

cv2.imshow("top 4 people living on the Earth", mosaic)
cv2.waitKey(0)

113